In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import numpy as np
from sklearn.metrics import f1_score
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, TrainingArguments, Trainer

tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [4]:
!unzip data.zip

Archive:  data.zip
replace __MACOSX/._test_data? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_data/113534_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/test_data/._113534_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_data/547532_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/test_data/._547532_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_data/204220_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/test_data/._204220_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_data/185911_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/test_data/._185911_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_data/194112_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import pandas as pd
import os
import re
# collect data

def create_dataset(txt_file, ann_file):
    with open(txt_file, "r") as txt_f, open(ann_file, "r") as ann_f:
        fulltext = re.sub('\xa0', ' ', txt_f.read())
        annotations = ann_f.readlines()
        
        sentences = [x for x in filter(None, fulltext.split('\n'))]
        labels = [list() for _ in sentences]
        for line in annotations:
            if line.startswith("T"):
                _, entity_type, start, end, *text = line.split()
                try:
                    start = int(start)
                    end = int(end)
                except Exception as e:
                    break
                for i, curr_sent in enumerate(sentences):
                    esc_text = re.escape(curr_sent)
                    for curr_sent_span in re.finditer(esc_text+'\n', fulltext):
                        curr_sent_span = curr_sent_span.span()
                        if end <= curr_sent_span[1]:
                            try:
                                labels[i].append((re.search(re.escape(" ".join(text)), curr_sent).span()[0],
                                                 re.search(re.escape(" ".join(text)), curr_sent).span()[1],
                                                 entity_type))
#                                 print(text,
#                                       re.search(re.escape(" ".join(text)), curr_sent).span()[0],
#                                       re.search(re.escape(" ".join(text)), curr_sent).span()[1],
#                                       entity_type)
                            except Exception as e:
                                continue
                            break
    return sentences, labels

X_data = []
y_data = []
numberoffiles = 0

for subdir, dirs, files in os.walk('/content/train_data'):
    for file in files:
        if file.endswith(".txt"):
            txt_file = os.path.join(subdir, file)
#             print(txt_file)
            filetext = open(txt_file, 'r').read()
            ann_file = os.path.join(txt_file[:-4] + ".ann")
            if os.path.isfile(ann_file):
                numberoffiles += 1
                X, y = create_dataset(txt_file, ann_file)
                X_data.extend(X)
                y_data.extend(y)

In [8]:
assert len(y_data) == len(X_data)
len(y_data)

7976

In [9]:
X_data, y_data

(['Марк Цукерберг стал Человеком года',
  'Журнал Time назвал Марка Цукерберга, основателя социальной сети Facebook с более чем половиной миллиарда пользователей по всему миру, Человеком года. По мнению редакции Time, в течение 2010 года Цукерберг существенным образом изменил жизнь миллионов людей и оказал огромное влияние на события, происходящие в мире. «Это не просто новая технология. Это социальная инженерия. Это меняет то, каким образом мы общаемся друг с другом. Я действительно думаю, что это влияет на человеческую природу так, как ещё никогда до этого», — сказал в эфире телеканала NBC главный редактор Time Ричард Стенгел.',
  '26-летний Цукерберг стал самым молодым победителем в этой номинации в истории — с тех пор, как в 1927 году первым Человеком года был назван пилот Чарльз Линдберг, в одиночку перелетевший Атлантический океан.',
  'Впрочем, решение относительно Цукерберга далось Time с большим трудом: победителем читательского голосования на сайте журнала стал основатель Wik

In [11]:
dataset = list(zip(X_data, y_data))

In [28]:
dataset

[('Марк Цукерберг стал Человеком года',
  [(0, 14, 'PERSON', 'B'), (20, 34, 'AWARD', 'B')]),
 ('Журнал Time назвал Марка Цукерберга, основателя социальной сети Facebook с более чем половиной миллиарда пользователей по всему миру, Человеком года. По мнению редакции Time, в течение 2010 года Цукерберг существенным образом изменил жизнь миллионов людей и оказал огромное влияние на события, происходящие в мире. «Это не просто новая технология. Это социальная инженерия. Это меняет то, каким образом мы общаемся друг с другом. Я действительно думаю, что это влияет на человеческую природу так, как ещё никогда до этого», — сказал в эфире телеканала NBC главный редактор Time Ричард Стенгел.',
  [(7, 11, 'ORGANIZATION', 'B'),
   (19, 35, 'PERSON', 'B'),
   (64, 72, 'PRODUCT', 'B'),
   (75, 104, 'NUMBER', 'B'),
   (7, 11, 'ORGANIZATION'),
   (175, 194, 'DATE'),
   (552, 555, 'ORGANIZATION'),
   (556, 572, 'PROFESSION'),
   (578, 592, 'PERSON'),
   (134, 148, 'AWARD'),
   (134, 148, 'AWARD'),
   (2

In [33]:
import torch
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments

def convert_to_bio_format(data):
    new_data = []
    for text, annotations in data:
        entities = sorted(annotations, key=lambda x: x[0])
        bio_labels = ["O"] * len(text)

        for entity in entities:
            start, end, label = entity[:3]

            bio_labels[start] = f"B-{label}"
            for i in range(start + 1, end):
                bio_labels[i] = f"I-{label}"

        new_data.append({"text": list(text), "labels": bio_labels})

    return new_data


# Prepare the dataset
class NERDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
      item = self.data[idx]
      text = item["text"]
      labels = item["labels"]
  
      encoding = self.tokenizer.encode_plus(
          text,
          truncation=True,
          max_length=self.max_len,
          padding="max_length",
          is_split_into_words=True,
          return_tensors="pt",
      )
  
      label_ids = [self.tokenizer.convert_tokens_to_ids(label) for label in labels]
      label_ids = label_ids[:self.max_len]  # Truncate labels if needed
      label_ids_padded = label_ids + [0] * (self.max_len - len(label_ids))
  
      return {
          "input_ids": encoding["input_ids"].squeeze(),
          "attention_mask": encoding["attention_mask"].squeeze(),
          "labels": torch.tensor(label_ids_padded, dtype=torch.long),
      }



        

train_data = convert_to_bio_format(dataset)

# Assuming that 80% of the data is used for training and 20% for validation
train_len = int(0.8 * len(train_data))
val_len = len(train_data) - train_len

train_dataset = NERDataset(train_data[:train_len], tokenizer)
val_dataset = NERDataset(train_data[train_len:], tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./ner_xlm_roberta")

IndexError: ignored

In [21]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    labels_flat = labels.flatten()
    preds_flat = preds.flatten()

    mask = labels_flat != -100  # Ignore -100 labels
    labels_flat = labels_flat[mask]
    preds_flat = preds_flat[mask]

    f1 = f1_score(labels_flat, preds_flat, average="weighted")
    return {"f1": f1}